# Pandas - Zpracování dat (část 2)

V tomto notebooku se naučíme:
- Používat `loc` k modifikaci dat
- Pracovat s prázdnými hodnotami (`isnull`, `notnull`, `dropna`)
- Lambda funkce
- Metodu `apply`

In [1]:
import pandas as pd

---
## 1. Použití `loc` k modifikaci dat

Metodu `loc` lze použít nejen k filtrování dat, ale také k jejich úpravě.

**Syntaxe:**
```python
df.loc[maska = podminka pro řádky, 'sloupec_k_upravě'] = 'nová_hodnota'
```

Kde:
- **podminka** - Series s hodnotami True/False
- **sloupec_k_upravě** - název sloupce, jehož hodnoty chceme změnit

> **Poznámka:** Pokud chceme upravit všechny řádky, místo podmínky napíšeme `:`. Použití `loc` mění data nevratně - je dobré si před úpravou vytvořit kopii pomocí `df.copy()`.

In [2]:
# Vytvoříme ukázkový DataFrame
data = {
    'date': ['2023-1', '1888-0', '2099-13', '2020-5'],
    'currency': ['zł', 'EUR', 'zł', 'EUR'],
    'value': [100, 50, 200, 75],
    'product_types': ['10pcs.', '10 pcs.', '5pcs.', '10pcs.']
}
df = pd.DataFrame(data)

In [3]:
print(df)

      date currency  value product_types
0   2023-1       zł    100        10pcs.
1   1888-0      EUR     50       10 pcs.
2  2099-13       zł    200         5pcs.
3   2020-5      EUR     75        10pcs.


### Příklad: Změna měny z 'zł' na 'PLN'

In [4]:
df.loc[df['currency'] ==  'zł', 'currency']

0    zł
2    zł
Name: currency, dtype: object

In [5]:
# Změna měny z lokální zkratky 'zł' na mezinárodní 'PLN'
df.loc[df['currency'] ==  'zł', 'currency'] = 'PLN'

In [6]:
df

,date,currency,value,product_types
0,2023-1,PLN,100,10pcs.
1,1888-0,EUR,50,10 pcs.
2,2099-13,PLN,200,5pcs.
3,2020-5,EUR,75,10pcs.


In [8]:
# Kontrola - neměly by zůstat žádné řádky s currency='zł'
df.loc[df['currency']=='zł']

,date,currency,value,product_types


### Otázka k zamyšlení
Proč je důležité před použitím `loc` k úpravě dat vytvořit kopii DataFrame?

### Úloha: Oprav chybu v kódu
Následující kód má za úkol změnit všechny hodnoty ve sloupci 'value' na dvojnásobek. Najdi a oprav chybu.

In [10]:
df_copy = df.copy()

In [11]:
# OPRAV CHYBU:
df.loc['value'] = df['value'] * 2

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 0 to value
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           4 non-null      object 
 1   currency       4 non-null      object 
 2   value          4 non-null      float64
 3   product_types  4 non-null      object 
dtypes: float64(1), object(3)
memory usage: 372.0+ bytes


In [22]:
df.loc['value']

date             NaN
currency         NaN
value            NaN
product_types    NaN
Name: value, dtype: object

In [14]:
df['value'] * 2

0        200.0
1        100.0
2        400.0
3        150.0
value      NaN
Name: value, dtype: float64

In [12]:
df

,date,currency,value,product_types
0,2023-1,PLN,100.0,10pcs.
1,1888-0,EUR,50.0,10 pcs.
2,2099-13,PLN,200.0,5pcs.
3,2020-5,EUR,75.0,10pcs.
value,NaN,NaN,NaN,NaN


In [13]:
df_copy

,date,currency,value,product_types
0,2023-1,PLN,100,10pcs.
1,1888-0,EUR,50,10 pcs.
2,2099-13,PLN,200,5pcs.
3,2020-5,EUR,75,10pcs.


In [19]:
df.loc[:,'value'] = df['value'] * 2

In [20]:
df

,date,currency,value,product_types
0,2023-1,PLN,200.0,10pcs.
1,1888-0,EUR,100.0,10 pcs.
2,2099-13,PLN,400.0,5pcs.
3,2020-5,EUR,150.0,10pcs.
value,NaN,NaN,NaN,NaN


---
## 2. Práce s prázdnými hodnotami

Prázdné hodnoty se v datech objevují často z různých důvodů:
- chyby při zadávání
- chybějící informace (legitimní prázdná hodnota)
- chyby při zpracování (např. problémy s převodem znaků na čísla)

Pandas poskytuje tyto metody pro práci s prázdnými hodnotami:
- `isnull()` / `isna()` - zjistí, zda je hodnota prázdná
- `notnull()` / `notna()` - zjistí, zda hodnota není prázdná
- `dropna()` - odstraní řádky s prázdnými hodnotami

In [24]:
# Vytvoříme DataFrame s prázdnými hodnotami
# import numpy as np

data_null = {
    'product_types': ['jablka', None, 'hrušky', 'banány', None],
    'product_line': [None, 'ovoce', None, None, 'zelenina'],
    'value': [100, 200, None, 150, 300]
}
df_null = pd.DataFrame(data_null)

In [25]:
print(df_null)

  product_types product_line  value
0        jablka         None  100.0
1          None        ovoce  200.0
2        hrušky         None    NaN
3        banány         None  150.0
4          None     zelenina  300.0


### Metody `isnull()` / `isna()`

Tyto metody jsou identické (sdílejí i dokumentaci). Vrací True tam, kde je hodnota prázdná.

In [26]:
# Zobrazení původního obsahu
df_null

,product_types,product_line,value
0,jablka,None,100.0
1,None,ovoce,200.0
2,hrušky,None,NaN
3,banány,None,150.0
4,None,zelenina,300.0


In [27]:
# Kontrola, které hodnoty jsou prázdné
df_null.isnull()

,product_types,product_line,value
0,False,True,False
1,True,False,False
2,False,True,True
3,False,True,False
4,True,False,False


In [28]:
df_null['product_types'].isnull()

0    False
1     True
2    False
3    False
4     True
Name: product_types, dtype: bool

In [29]:
# Kontrola prázdných hodnot pouze ve sloupci product_line
df_null['product_line'].isnull()

0     True
1    False
2     True
3     True
4    False
Name: product_line, dtype: bool

In [31]:
df_null.isnull().sum()

product_types    2
product_line     3
value            1
dtype: int64

In [32]:
df_null.isnull().sum().sum()

np.int64(6)

In [34]:
df_null.isnull().any()

product_types    True
product_line     True
value            True
dtype: bool

In [37]:
df_null.isnull().any(axis = 1)

0    True
1    True
2    True
3    True
4    True
dtype: bool

### Metoda `dropna()`

Odstraňuje řádky s prázdnými hodnotami z DataFrame.

**Parametry:**
- `how` - kdy odstranit řádek:
  - `'any'` - řádek se odstraní, pokud obsahuje jakoukoliv prázdnou hodnotu
  - `'all'` - řádek se odstraní pouze pokud jsou všechny hodnoty prázdné
- `subset` - seznam sloupců, na základě kterých se řádky mažou

In [38]:
df_null

,product_types,product_line,value
0,jablka,None,100.0
1,None,ovoce,200.0
2,hrušky,None,NaN
3,banány,None,150.0
4,None,zelenina,300.0


In [39]:
df_null.dropna(subset=['product_line'])

,product_types,product_line,value
1,None,ovoce,200.0
4,None,zelenina,300.0


In [40]:
df_null.dropna(subset=['value'])

,product_types,product_line,value
0,jablka,None,100.0
1,None,ovoce,200.0
3,banány,None,150.0
4,None,zelenina,300.0


In [43]:
df_null.dropna(subset=['product_line', 'value'], how = 'any')

,product_types,product_line,value
1,None,ovoce,200.0
4,None,zelenina,300.0


In [44]:
df_null.dropna(subset=['product_line', 'value'], how = 'all')

,product_types,product_line,value
0,jablka,None,100.0
1,None,ovoce,200.0
3,banány,None,150.0
4,None,zelenina,300.0


### Otázka k zamyšlení
Jaký je rozdíl mezi `how='any'` a `how='all'` v metodě `dropna()`? Kdy bys použil/a kterou variantu?

### Úloha: Doplň kód
Doplň kód tak, aby odstranil řádky, které mají prázdnou hodnotu ve sloupci 'product_types'.

In [ ]:
# DOPLŇ:
df_cleaned = df_null._

In [46]:
# DOPLŇ:
df_cleaned = df_null.dropna(subset = 'product_types')

---
## 3. Lambda funkce

Lambda je "mini-funkce" převzatá z jazyka LISP. Má stejný efekt jako běžná funkce, ale se zkrácenou syntaxí:
1. Nemá závorky kolem argumentů
2. Nemá klíčové slovo `return`

Lambda funkce nemá jméno, ale může být přiřazena do proměnné.

In [48]:
# Klasická implementace funkce
def f(x):
    return x**2

In [50]:
f(4)

16

In [52]:
f.__class__

function

In [55]:
# Implementace pomocí lambda (místo def píšeme lambda)
# a přiřazení do proměnné
neviem_co_stym = lambda x: x**2

In [57]:
neviem_co_stym(4)

16

In [60]:
# Jednorázové použití lambda
(lambda x: x**3)(4)

64

### Úloha: Oprav chybu
Následující lambda funkce má počítat součet dvou čísel. Najdi a oprav chybu.

In [61]:
# OPRAV CHYBU:
soucet = lambda x, y: return x + y

SyntaxError: invalid syntax (1757379415.py, line 2)

In [62]:
# OPRAV CHYBU:
soucet = lambda x, y: x + y

In [63]:
soucet(5,-2)

3

---
## 4. Metoda `apply`

Metoda `apply` umožňuje aplikovat funkci na sloupce nebo řádky DataFrame.

**Parametry:**
- `func` - funkce, kterou chceme použít (může být lambda)
- `axis` - osa, na které se funkce aplikuje:
  - `0` - funkce se aplikuje na všechny sloupce
  - `1` - funkce se aplikuje na řádky

> **Poznámka:** Iterování přes DataFrame pomocí `for` cyklu je neefektivní - používejte raději `apply`.

In [64]:
# Ukázkový DataFrame
data_apply = {
    'value': [100, 200, 300, 400],
    'province': ['PRAHA', 'BRNO', 'OSTRAVA', 'PLZEŇ']
}
df_apply = pd.DataFrame(data_apply)

In [65]:
print(df_apply)

   value province
0    100    PRAHA
1    200     BRNO
2    300  OSTRAVA
3    400    PLZEŇ


### Příklad: Zvýšení hodnoty ve sloupci o 23%

In [66]:
df_copy = df_apply.copy()

In [67]:
df_apply.loc[:, 'value'] = df_apply['value'] * 1.23

In [68]:
df_apply

,value,province
0,123,PRAHA
1,246,BRNO
2,369,OSTRAVA
3,492,PLZEŇ


In [69]:
# Zvýšení čísla ve sloupci value o 23%
df_apply = df_copy.copy()
df_apply

,value,province
0,100,PRAHA
1,200,BRNO
2,300,OSTRAVA
3,400,PLZEŇ


In [71]:
df_apply[['value']].apply(lambda x: 1.23 * x)

,value
0,123.0
1,246.0
2,369.0
3,492.0


In [73]:
df_apply['value_added'] = df_apply[['value']].apply(lambda x: 1.23 * x)

In [74]:
df_apply

,value,province,value_added
0,100,PRAHA,123.0
1,200,BRNO,246.0
2,300,OSTRAVA,369.0
3,400,PLZEŇ,492.0


### Příklad: Převod na malá písmena

In [75]:
# Změna písmen na malá ve sloupci province
df_apply['province'].apply(lambda x: x.lower())

0      praha
1       brno
2    ostrava
3      plzeň
Name: province, dtype: object

### Úloha: Doplň kód
Doplň kód tak, aby převedl všechna písmena ve sloupci 'province' na velká.

In [77]:
# DOPLŇ:
df_apply['province'].apply(lambda x: x.capitalize())

0      Praha
1       Brno
2    Ostrava
3      Plzeň
Name: province, dtype: object

In [79]:
df_apply['province'] = df_apply['province'].apply(lambda x: x.capitalize())

In [80]:
df_apply

,value,province,value_added
0,100,Praha,123.0
1,200,Brno,246.0
2,300,Ostrava,369.0
3,400,Plzeň,492.0


---
## Cvičení 1: Odstranění chyb

Použij soubor **product_prices_renamed.csv** a pomocí `loc` oprav následující chyby:

1. Ve sloupci **date** se objevila data z roku 1888 - '1888-0', změň hodnotu na 1999-1
2. Ve sloupci **date** se objevila data z roku 2099 - '2099-13', změň hodnotu na 2019-1
3. Ve sloupci **product_types** je překlep - oprav ho. Počet kusů má být '10pcs.'. Zkontroluj, zda byl úkol splněn.
4. Pomocí `loc` převeď hodnoty zadané v `EUR` na `PLN` s kurzem 4.15
5. Odfiltruj z datasetu řádky, kde je cena produktu 3000.

**Tip:** Místo psaní `loc` dvakrát nejdřív vyfiltruj data pro řádky kde **currency** = `EUR` a ulož do proměnné.

> Pamatuj, že `loc` mění data nevratně.

In [82]:
# Tvůj kód zde:
df = pd.read_csv('../Data/product_prices_renamed.csv', sep = ';')
df.columns

Index(['province', 'product_types', 'currency', 'product_group_id',
       'product_line', 'value', 'date'],
      dtype='object')

In [83]:
df.shape

(149940, 7)

In [84]:
df.loc[df['date'] == '1888-0', 'date']

11908     1888-0
14765     1888-0
16125     1888-0
18737     1888-0
20293     1888-0
33038     1888-0
50542     1888-0
51536     1888-0
85091     1888-0
95997     1888-0
100513    1888-0
110578    1888-0
111306    1888-0
123125    1888-0
137245    1888-0
144334    1888-0
147385    1888-0
Name: date, dtype: object

In [87]:
# 1. samotna oprava 
df.loc[df['date'] == '1888-0', 'date'] = '1999-1'

In [86]:
# kontrola 
df.loc[df['date'] == '1888-0', 'date']

Series([], Name: date, dtype: object)

In [ ]:
# Ve sloupci date se objevila data z roku 2099 - '2099-13', změň hodnotu na 2019-1
# 2 Oprava data 2099-13 na 2019-1

In [89]:
df.loc[df['date'] == '2099-13', 'date'] = '2019-1'

In [90]:
# kontrola
df.loc[df['date'] == '2099-13', 'date']

Series([], Name: date, dtype: object)

In [91]:
# 3. Ve sloupci product_types je překlep - oprav ho. Počet kusů má být '10pcs.'. Zkontroluj, zda byl úkol splněn.
df['product_types'].unique()

array([nan, 'whole pickled cucumbers 0.9l - per 1pc.',
       'fresh chichen egges - per 666pcs.',
       '30% tomato concentrate - per 1kg',
       'frozen carrot and pea mix - per 1kg',
       'beet sugar white, bagged - per 1kg',
       'apple juice, boxed - per 1l', 'white table salt bagged - per 1kg',
       'natural chocolate plain - per 1kg'], dtype=object)

In [92]:
# maska: df['product_types'] == 'fresh chichen egges - per 666pcs.'
df.loc[df['product_types'] == 'fresh chichen egges - per 666pcs.', 'product_types']

6         fresh chichen egges - per 666pcs.
13        fresh chichen egges - per 666pcs.
38        fresh chichen egges - per 666pcs.
202       fresh chichen egges - per 666pcs.
229       fresh chichen egges - per 666pcs.
                        ...                
149691    fresh chichen egges - per 666pcs.
149718    fresh chichen egges - per 666pcs.
149724    fresh chichen egges - per 666pcs.
149767    fresh chichen egges - per 666pcs.
149838    fresh chichen egges - per 666pcs.
Name: product_types, Length: 4284, dtype: object

In [93]:
df.loc[df['product_types'] == 'fresh chichen egges - per 666pcs.', 'product_types'] = 'fresh chichen egges - per 10pcs.'

In [94]:
df.loc[df['product_types'] == 'fresh chichen egges - per 666pcs.', 'product_types']

Series([], Name: product_types, dtype: object)

In [95]:
# Pomocí loc převeď hodnoty zadané v EUR na PLN s kurzem 4.15
df.columns

Index(['province', 'product_types', 'currency', 'product_group_id',
       'product_line', 'value', 'date'],
      dtype='object')

In [105]:
mask = df['currency'] == 'EUR'

In [103]:
df_copy = df.copy()

In [109]:
df.loc[mask, 'value'] = df.loc[mask, 'value'] * 4.15

In [111]:
df.loc[mask, 'currency'] = 'PLN'

In [112]:
# Odfiltruj z datasetu řádky, kde je cena produktu 3000.

In [115]:
df = df.loc[df['value'] != 3000]
df

,province,product_types,currency,product_group_id,product_line,value,date
0,SUBCARPATHIA,NaN,PLN,2,pork ham cooked - per 1kg,21.37,2013-3
1,ŁÓDŹ,NaN,PLN,4,bread - per 1kg,NaN,2018-2
2,KUYAVIA-POMERANIA,NaN,PLN,2,barley groats sausage - per 1kg,3.55,2019-12
3,LOWER SILESIA,NaN,PLN,2,dressed chickens - per 1kg,6.14,2019-2
4,WARMIA-MASURIA,NaN,PLN,2,Italian head cheese - per 1kg,5.63,2002-3
...,...,...,...,...,...,...,...
149935,KUYAVIA-POMERANIA,NaN,PLN,2,pork meat (raw bacon) - per 1kg,12.15,2016-11
149936,ŁÓDŹ,"beet sugar white, bagged - per 1kg",PLN,3,NaN,0.00,2012-5
149937,LESSER POLAND,NaN,PLN,4,plain mixed bread (wheat-rye) - per 1kg,3.05,2008-6
149938,WARMIA-MASURIA,NaN,PLN,2,boneless beef (sirloin) - per 1kg,11.87,2000-11


---
## Cvičení 2: Přidání sloupce

Navazuje na řešení předchozího cvičení. Vytvoř nový sloupec **product** pomocí sloupců **product_types** a **product_line**:

1. Zkontroluj, že sloupce **product_types** a **product_line** jsou komplementární (prázdná hodnota v jednom sloupci znamená neprázdnou hodnotu v druhém)
2. Vytvoř nový sloupec **product** s hodnotami ze sloupce **product_types**: `df['product'] = df['product_types']`
3. Najdi neprázdné hodnoty ve sloupci **product_line** a vlož je do sloupce **product**
4. Zvolenou metodou zkontroluj, zda jsou všechny hodnoty ve sloupci **product** neprázdné
5. Odstraň duplicity z tabulky
6. Pomocí metody `to_csv` ulož data (budeme je používat později), nastav `sep=';'` a `index=False`. Ulož soubor jako `product_prices_cleaned.csv`

**Ukázka volání:**
```python
df.to_csv(
    'filepath',
    sep=';',  # nastavení oddělovače
    index=False
)
```

In [ ]:
# Tvůj kód zde:


---
## Přehled použitých metod a funkcí

| Metoda/Funkce | Popis |
|--------------|-------|
| `df.loc[podminka, sloupec]` | Výběr/úprava dat podle podmínky a sloupce |
| `df.copy()` | Vytvoření kopie DataFrame |
| `df.isnull()` / `df.isna()` | Kontrola, zda jsou hodnoty prázdné (vrací True/False) |
| `df.notnull()` / `df.notna()` | Kontrola, zda hodnoty nejsou prázdné |
| `df.dropna(subset=[...])` | Odstranění řádků s prázdnými hodnotami |
| `df.shape` | Vrací rozměry DataFrame (řádky, sloupce) |
| `lambda x: výraz` | Anonymní funkce pro jednoduché transformace |
| `df.apply(func)` | Aplikace funkce na sloupce nebo řádky |
| `df.drop_duplicates()` | Odstranění duplicitních řádků |
| `df.to_csv(path, sep, index)` | Uložení DataFrame do CSV souboru |

---
## Důležité: Jak metódy mění DataFrame?

### 1. Metody s parametrem `inplace`

Mnoho metod má parametr `inplace`, který určuje, zda se změní původní DataFrame:

**Bez `inplace` (výchozí `inplace=False`):**
- Vrací **nový DataFrame**
- Původní DataFrame zůstává **beze změny**
- Výsledek je třeba uložit do proměnné

```python
df_novy = df.dropna()  # df zůstává nezměněn
df_novy = df.drop_duplicates()  # df zůstává nezměněn
```

**S `inplace=True`:**
- Mění **přímo původní DataFrame**
- Vrací `None` (ne nový DataFrame!)
- Změna je nevratná

```python
df.dropna(inplace=True)  # df se změní přímo
df.drop_duplicates(inplace=True)  # df se změní přímo
```

**Metody s parametrem `inplace`:**
- `dropna()`
- `drop_duplicates()`
- `reset_index()`
- `sort_values()`
- `rename()`
- `fillna()`
- `replace()`

### 2. Metody BEZ parametru `inplace`

**`loc` - vždy mění přímo:**
```python
df.loc[podminka, 'sloupec'] = hodnota  # MĚNÍ PŮVODNÍ df!
```
- Nemá parametr `inplace`
- Vždy mění původní DataFrame nevratně
- Proto je dobré před použitím vytvořit kopii: `df = df.copy()`

**`apply()` - vždy vrací nový objekt:**
```python
vysledek = df.apply(funkce)  # VRACÍ nový DataFrame/Series
```
- Nemá parametr `inplace`
- Vždy vrací nový DataFrame nebo Series
- Původní DataFrame zůstává nezměněn

**Filtrování - vrací nový DataFrame:**
```python
df_filtrovany = df[df['A'] > 5]  # VRACÍ nový DataFrame
```

### 3. Doporučení pro začátečníky

**Bezpečný přístup - explicitní přiřazení:**
```python
df = df.dropna()  # Jasné: pracuješ s novým DataFrame
df = df.drop_duplicates()  # Jasné: vytváříš nový DataFrame
```

**Nebo explicitní `inplace`:**
```python
df.dropna(inplace=True)  # Jasné: měníš původní DataFrame
df.drop_duplicates(inplace=True)  # Jasné: měníš původní DataFrame
```

**Pozor na `loc` - vždy vytvoř kopii před úpravami:**
```python
df_backup = df.copy()  # Záloha před změnami
df.loc[podminka, 'sloupec'] = nova_hodnota  # Mění df!
```